In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Constants
UPBIT_URL = "https://api.upbit.com/v1/candles/days"
MARKET_CODE = "KRW-ARB"  # Assuming ARB is traded against KRW, adjust if necessary
DAYS_COUNT = 600

# Function to fetch daily data
def fetch_daily_data(market_code, days):
    end_date = datetime.now()
    all_data = []

    while len(all_data) < days:
        params = {
            "market": market_code,
            "to": end_date.strftime('%Y-%m-%dT%H:%M:%S'),
            "count": min(days - len(all_data), 200)  # API may have limits, adjust accordingly
        }
        response = requests.get(UPBIT_URL, params=params)
        data = response.json()

        if not data:
            break

        all_data.extend(data)
        # Update end_date to fetch previous data in next iteration
        last_date = datetime.strptime(data[-1]['candle_date_time_utc'], '%Y-%m-%dT%H:%M:%S')
        end_date = last_date - timedelta(seconds=1)

    return all_data[:days]

# Fetch the data
data = fetch_daily_data(MARKET_CODE, DAYS_COUNT)

# Convert to DataFrame for easier handling
df = pd.DataFrame(data)

# Selecting relevant columns and renaming for clarity
df_relevant = df[['candle_date_time_kst', 'opening_price', 'high_price', 'low_price', 'trade_price', 'candle_acc_trade_volume']].copy()
df_relevant.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

# Save to CSV
csv_file_path = 'upbit_arb_60_days.csv'
df_relevant.to_csv(csv_file_path, index=False)

print(f"Data saved to {csv_file_path}")


Data saved to upbit_arb_60_days.csv
Reversed data saved to upbit_arb_60_days_reversed.csv


In [ ]:
# 데이터 프레임을 역순으로 정렬
df_reversed = df_relevant.iloc[::-1].reset_index(drop=True)

# 역순으로 정렬된 데이터를 새로운 CSV 파일에 저장
reversed_csv_file_path = 'upbit_arb_60_days_reversed.csv'
df_reversed.to_csv(reversed_csv_file_path, index=False)

print(f"Reversed data saved to {reversed_csv_file_path}")